# Pandas & Financial Data Analysis

#### Market data dashboard:
- Download data for multiple stocks using yfinance
- Calculate daily returns and volatility
- Generate performance summary using pandas

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define stock tickers and date range
tickers = ['AAPL', 'MSFT', 'GOOGL', 'WMT']
start_date = '2023-01-01'
end_date = '2024-01-01'

# Download adjusted close prices for all tickers
data = yf.download(tickers, start=start_date, end=end_date)['Close']

# Calculate daily returns
daily_returns = data.pct_change().dropna()

# Compute volatility (daily and annualized)
daily_volatility = daily_returns.std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Generate performance summary
performance_summary = pd.DataFrame({
    'Mean Daily Return': daily_returns.mean(),
    'Daily Volatility': daily_volatility,
    'Annualized Volatility': annualized_volatility,
    'Max Daily Return': daily_returns.max(),
    'Min Daily Return': daily_returns.min()
})

print("Market Data Dashboard Performance Summary:")
print(performance_summary)



[*********************100%***********************]  4 of 4 completed

Market Data Dashboard Performance Summary:
        Mean Daily Return  Daily Volatility  Annualized Volatility  \
Ticker                                                               
AAPL             0.001835          0.012570               0.199543   
GOOGL            0.001990          0.019149               0.303982   
MSFT             0.001972          0.015824               0.251203   
WMT              0.000485          0.009844               0.156262   

        Max Daily Return  Min Daily Return  
Ticker                                      
AAPL            0.046927         -0.048020  
GOOGL           0.072787         -0.095094  
MSFT            0.072434         -0.043743  
WMT             0.024499         -0.080928  


#### Technical analysis system:
- Calculate various moving averages
- Generate trading signals
- Backtest strategy performance

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker and date range
ticker = 'AAPL'
start_date = '2023-01-01'
end_date = '2024-01-01'

# Download historical data
df = yf.download(ticker, start=start_date, end=end_date)

# Calculate moving averages
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# Generate trading signals: 1 (Buy) when short MA > long MA; -1 (Sell) otherwise
df['Signal'] = np.where(df['SMA_5'] > df['SMA_20'], 1, -1)

# Backtest strategy: Shift signal by 1 to avoid lookahead bias and multiply by daily return
df['Daily Return'] = df['Close'].pct_change()
df['Strategy Return'] = df['Signal'].shift(1) * df['Daily Return']
df['Cumulative Strategy Return'] = (1 + df['Strategy Return']).cumprod()
df['Cumulative Market Return'] = (1 + df['Daily Return']).cumprod()

print("Technical Analysis System (Last 10 Days):")
print(df[['Close', 'SMA_5', 'SMA_20', 'Signal', 'Cumulative Strategy Return', 'Cumulative Market Return']].tail(10))



[*********************100%***********************]  1 of 1 completed

Technical Analysis System (Last 10 Days):
Price            Close       SMA_5      SMA_20 Signal  \
Ticker            AAPL                                  
Date                                                    
2023-12-15  196.390900  195.134433  191.375502      1   
2023-12-18  194.720901  195.673199  191.683651      1   
2023-12-19  195.764648  196.116534  191.956513      1   
2023-12-20  193.667252  195.494272  192.164763      1   
2023-12-21  193.518127  194.812366  192.332257      1   
2023-12-22  192.444595  194.023105  192.512675      1   
2023-12-26  191.897873  193.458499  192.674702      1   
2023-12-27  191.997269  192.705023  192.811382     -1   
2023-12-28  192.424713  192.456516  193.020627     -1   
2023-12-29  191.380966  192.029083  193.148857     -1   

Price      Cumulative Strategy Return Cumulative Market Return  
Ticker                                                          
Date                                                            
2023-12-15           

#### Portfolio analysis tool:
- Track multiple stocks performance
- Calculate portfolio statistics
- Generate risk metrics (Beta, Sharpe Ratio)

In [16]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define portfolio tickers, allocations, and date range
tickers = ['AAPL', 'MSFT', 'GOOGL', 'WMT']
weights = np.array([0.3, 0.3, 0.2, 0.2])
start_date = '2023-01-01'
end_date = '2024-01-01'

# Download adjusted close prices for portfolio stocks
data = yf.download(tickers, start=start_date, end=end_date)['Close']
# Calculate daily returns for portfolio stocks
returns = data.pct_change().dropna()

# Calculate portfolio daily returns as weighted sum
portfolio_returns = (returns * weights).sum(axis=1)

# Download benchmark (S&P 500) data and calculate daily returns
spy = yf.download('^GSPC', start=start_date, end=end_date)['Close']
spy_returns = spy.pct_change().dropna()

# Align dates between portfolio and benchmark
common_dates = portfolio_returns.index.intersection(spy_returns.index)
portfolio_returns = portfolio_returns.loc[common_dates]
spy_returns = spy_returns.loc[common_dates]

# Convert both series to 1D numpy arrays
port_arr = portfolio_returns.to_numpy().flatten()
spy_arr = spy_returns.to_numpy().flatten()

# Check if both arrays have the same length
if len(port_arr) != len(spy_arr):
    raise ValueError("Mismatch in the length of portfolio and benchmark returns after alignment.")

# Calculate Beta: covariance(portfolio, benchmark) / variance(benchmark)
covariance = np.cov(port_arr, spy_arr)[0, 1]
beta = covariance / np.var(spy_arr)

# Calculate annualized risk metrics for the portfolio
annualized_return = portfolio_returns.mean() * 252
annualized_volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratio = (annualized_return) / annualized_volatility  # Assuming risk-free rate ~ 0

# Prepare portfolio statistics summary
portfolio_stats = pd.DataFrame({
    'Annualized Return': [annualized_return],
    'Annualized Volatility': [annualized_volatility],
    'Sharpe Ratio': [sharpe_ratio],
    'Beta': [beta]
})

print("Portfolio Analysis Tool:")
print(portfolio_stats)


[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed

Portfolio Analysis Tool:
   Annualized Return  Annualized Volatility  Sharpe Ratio      Beta
0           0.412948               0.175669      2.350713  1.066479


#### Market sentiment analyzer:
- Download index data (S&P 500, NASDAQ)
- Calculate market breadth indicators
- Generate market health report

In [13]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define market indices and date range
indices = ['^GSPC', '^IXIC']  # S&P 500 and NASDAQ
start_date = '2023-01-01'
end_date = '2024-01-01'

# Download adjusted close prices for indices
index_data = yf.download(indices, start=start_date, end=end_date)['Close']
index_returns = index_data.pct_change().dropna()

# Market breadth indicator: percentage of days with positive returns for each index
breadth = (index_returns > 0).mean() * 100

# Generate market health report
market_health = pd.DataFrame({
    'Average Daily Return': index_returns.mean().values,
    'Daily Volatility': index_returns.std().values,
    'Positive Days (%)': breadth.values
}, index=indices)

print("Market Sentiment Analyzer:")
print(market_health)


[*********************100%***********************]  2 of 2 completed

Market Sentiment Analyzer:
       Average Daily Return  Daily Volatility  Positive Days (%)
^GSPC              0.000922          0.008258          55.020080
^IXIC              0.001540          0.010958          56.626506


#### Dividend analysis tool:
- Download dividend history
- Calculate dividend growth rates
- Project future dividend income

In [14]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the stock ticker
ticker = 'AAPL'
t = yf.Ticker(ticker)
dividends = t.dividends

if len(dividends) < 2:
    print("Not enough dividend data to calculate growth rate.")
else:
    # Calculate dividend CAGR (Compound Annual Growth Rate)
    first_dividend = dividends.iloc[0]
    last_dividend = dividends.iloc[-1]
    num_years = (dividends.index[-1] - dividends.index[0]).days / 365.25
    dividend_cagr = (last_dividend / first_dividend) ** (1 / num_years) - 1

    # Project dividend income for next 5 years based on CAGR
    future_dividends = [last_dividend * ((1 + dividend_cagr) ** year) for year in range(1, 6)]
    
    dividend_report = pd.DataFrame({
        'Year': list(range(1, 6)),
        'Projected Dividend': future_dividends
    })
    
    print(f"Dividend Analysis for {ticker}:")
    print(f"Dividend CAGR: {dividend_cagr:.2%}")
    print(dividend_report)


Dividend Analysis for AAPL:
Dividend CAGR: 17.68%
   Year  Projected Dividend
0     1            0.294189
1     2            0.346189
2     3            0.407380
3     4            0.479388
4     5            0.564123


#### Sector rotation analysis
- Download sector ETF data
- Compare sector performance
- Identify sector trends

In [15]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define sector ETFs and date range
sector_etfs = ['XLK', 'XLF', 'XLE', 'XLY', 'XLI', 'XLV']  # Tech, Finance, Energy, Consumer Discretionary, Industrials, Healthcare
start_date = '2023-01-01'
end_date = '2024-01-01'

# Download adjusted close prices for sector ETFs
sector_data = yf.download(sector_etfs, start=start_date, end=end_date)['Close']
sector_returns = sector_data.pct_change().dropna()

# Calculate cumulative return for each sector ETF over the period
cumulative_returns = (sector_data.iloc[-1] / sector_data.iloc[0] - 1) * 100

sector_analysis = pd.DataFrame({
    'Cumulative Return (%)': cumulative_returns.values
}, index=sector_etfs).sort_values(by='Cumulative Return (%)', ascending=False)

print("Sector Rotation Analysis:")
print(sector_analysis)


[*********************100%***********************]  6 of 6 completed

Sector Rotation Analysis:
     Cumulative Return (%)
XLY              57.511988
XLV              40.476761
XLE              17.877255
XLF              11.639237
XLK               2.984282
XLI               2.360457
